### Verifying the paths are working

In [56]:
from pathlib import Path
import os


# Base directory - assumes your notebook is in /notebooks
base_dir = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
print(f"Base directory: {base_dir}\nExists: {base_dir.exists()}\n")

# Data directories
data_dir = base_dir / '/Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data'
raw_data_dir = data_dir / '/Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data/raw'
processed_data_dir = data_dir / '/Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data/processed'
features_dir = data_dir / '/Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data/features'

print(f"Data directory: {data_dir}\nExists: {data_dir.exists()}")
print(f"Raw data directory: {raw_data_dir}\nExists: {raw_data_dir.exists()}")
print(f"Processed data directory: {processed_data_dir}\nExists: {processed_data_dir.exists()}")
print(f"Features directory: {features_dir}\nExists: {features_dir.exists()}\n")

# Path to metadata CSV
metadata_path = raw_data_dir / 'Birds Voice.csv'
print(f"Metadata path: {metadata_path}\nExists: {metadata_path.exists()}\n")

# Path to audio files
audio_files_dir = raw_data_dir / 'Voice of Birds'
print(f"Audio files directory: {audio_files_dir}\nExists: {audio_files_dir.exists()}\n")

# List bird folders if audio directory exists
if audio_files_dir.exists():
    bird_folders = [d for d in audio_files_dir.iterdir() if d.is_dir()]
    print(f"Found {len(bird_folders)} bird folders")
    if bird_folders:
        print("\nFirst few bird folders:")
        for bird_folder in bird_folders[:3]:  # Print first 3 as samples
            print(f"- {bird_folder.name}")
        
        # Check MP3 files in first bird folder
        sample_bird = bird_folders[0]
        mp3_files = list(sample_bird.glob('*.mp3'))
        print(f"\nFound {len(mp3_files)} MP3 files in {sample_bird.name}")
        if mp3_files:
            print("\nFirst few MP3 files:")
            for mp3 in mp3_files[:3]:  # Print first 3 as samples
                print(f"- {mp3.name}")
else:
    print("Warning: Audio files directory not found!")

Base directory: /Users/arnaumartin/Birdify
Exists: True

Data directory: /Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data
Exists: True
Raw data directory: /Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data/raw
Exists: True
Processed data directory: /Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data/processed
Exists: True
Features directory: /Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data/features
Exists: True

Metadata path: /Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data/raw/Birds Voice.csv
Exists: True

Audio files directory: /Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data/raw/Voice of Birds
Exists: True

Found 119 bird folders

First few bird folders:
- blue-throated_piping_guan_sound
- Andean Tinamou_sound
- highland_tinamou_sound

Found 6 MP3 files in blue-throated_piping_guan_sound

First few MP3 files:
- XC228153.mp3
- XC449253.mp3
- XC25

### Preprocessing audio files

In [55]:
import os
import essentia.standard as ess
import numpy as np
import pandas as pd
import glob
import random
from tqdm import tqdm
import warnings
from pathlib import Path
from collections import defaultdict
from audiomentations import Compose, TimeStretch, AddGaussianNoise, Shift
from scipy.signal import butter, lfilter
import noisereduce as nr
import requests

warnings.filterwarnings("ignore", category=RuntimeWarning)

# Configuration
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = BASE_DIR / '/Users/arnaumartin/Desktop/3r_uni/Taller tecnologia musical/Birdify/data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
METADATA_PATH = RAW_DATA_DIR / 'Birds Voice.csv'
AUDIO_FILES_DIR = RAW_DATA_DIR / 'Voice of Birds'
OUTPUT_DIR = PROCESSED_DATA_DIR / 'birdcall_segments_5s'
os.makedirs(OUTPUT_DIR, exist_ok=True)

LATAM_COUNTRIES = {
    'Argentina', 'Belize', 'Bolivia', 'Brazil', 'Chile',
    'Colombia', 'Costa Rica', 'Cuba', 'Dominican Republic',
    'Ecuador', 'El Salvador', 'Guatemala', 'Honduras', 'Mexico',
    'Nicaragua', 'Panama', 'Paraguay', 'Peru', 'Puerto Rico',
    'Uruguay', 'Venezuela'
}
VALID_VOCALIZATIONS = {'song', 'call', 'alarm call'}


# Read metadata and filtering
df = pd.read_csv(METADATA_PATH)
df['TYPE'] = df['TYPE'].str.lower()
df['Country'] = df['Country'].str.strip()
df = df[df['Country'].isin(LATAM_COUNTRIES) & df['TYPE'].isin(VALID_VOCALIZATIONS)]

# Download only if not already downloaded
for idx, row in df.iterrows():
    folder_name = row['common_name'].strip().replace(' ', '_').lower() + '_sound'
    folder_path = AUDIO_FILES_DIR / folder_name
    os.makedirs(folder_path, exist_ok=True)
    audio_url = row['Download_link']
    audio_filename = f"{row['xc_id']}.mp3"
    audio_path = folder_path / audio_filename
    if not audio_path.exists():
        try:
            r = requests.get(audio_url, timeout=20)
            if r.status_code == 200:
                with open(audio_path, 'wb') as f:
                    f.write(r.content)
                print(f"Downloading: {audio_filename} in {folder_name}")
            else:
                print(f"Error downloading {audio_url}: status {r.status_code}")
        except Exception as e:
            print(f"Error downloading {audio_url}: {e}")
    else:
        print(f"Already exists: {audio_filename} in {folder_name}")

# Audio processing parameters
fs = 22050
segment_duration = 5  # seconds
segment_samples = int(fs * segment_duration)
windowSize = 4096
hopSize = 2048
NRG_threshold_ratio = 0.01
required_segments = 50
target_bird_count = 10


# Define functions

def load_and_filter_metadata():
    df = pd.read_csv(METADATA_PATH, sep=",")
    df['TYPE'] = df['TYPE'].str.lower()
    df['Country'] = df['Country'].str.strip()
    latam_filter = df['Country'].isin(LATAM_COUNTRIES)
    vocalization_filter = df['TYPE'].isin(VALID_VOCALIZATIONS)
    filtered_df = df[latam_filter & vocalization_filter].copy()
    species_counts = filtered_df['common_name'].value_counts().reset_index()
    species_counts.columns = ['common_name', 'observation_count']
    top_species = species_counts.head(50)['common_name'].tolist()
    return filtered_df[filtered_df['common_name'].isin(top_species)]

def get_bird_folders_for_species(metadata_df):
    metadata_df['folder_name'] = metadata_df['common_name'].str.replace(' ', '_').str.lower() + '_sound'
    print("Carpetas esperadas:")
    print(metadata_df['folder_name'].unique())
    print("\nCarpetas reales en Voice of Birds:")
    print([f.name for f in AUDIO_FILES_DIR.glob("*") if f.is_dir()])
    existing_folders = []
    for folder in metadata_df['folder_name'].unique():
        full_path = AUDIO_FILES_DIR / folder
        if full_path.exists():
            existing_folders.append((folder.replace('_sound', ''), folder))
    return existing_folders


# Audio processing functions
def amplitude_normalization(x):
    max = np.max(np.abs(x))
    return x / max


def bandpass_filter(x, fs, low_freq=500, high_freq=9000, order=4):
    nyquist = 0.5 * fs
    low = low_freq / nyquist
    high = high_freq / nyquist
    b, a = butter(order, [low, high], btype='band')
    return lfilter(b, a, x)

def audio_preprocessing(x, fs):
    # Filtering
    x = bandpass_filter(x, fs)

    # Denoising
    x = nr.reduce_noise(y=x, sr=fs)
    
    # Amplitude normalization
    x = amplitude_normalization(x)
    
    return x

def safe_mono_loader(filepath, sample_rate):
    try:
        return ess.MonoLoader(filename=str(filepath), sampleRate=sample_rate)()
    except Exception as e:
        print(f"Warning: Could not load {filepath.name} - {str(e)}")
        return None

# Segmentation and preprocessing
def extract_birdcall_segments(x, params):
    if x is None or len(x) < params['segment_samples']:
        return []

    NRG = []
    for frame in ess.FrameGenerator(x, frameSize=params['windowSize'], 
                                  hopSize=params['hopSize'], startFromZero=True):
        NRG.append(ess.Energy()(frame))
    
    if not NRG:
        return []

    NRG = np.array(NRG)
    NRG = NRG / (np.max(NRG) + 1e-10)
    active_frames = np.where(NRG > params['NRG_threshold_ratio'])[0]

    if len(active_frames) == 0:
        return []

    segments = []
    for frame_idx in active_frames:
        sample_idx = frame_idx * params['hopSize']
        start = max(0, sample_idx - params['segment_samples'] // 2)
        end = start + params['segment_samples']
        if end > len(x):
            continue
        segment = x[start:end]
        if np.max(np.abs(segment)) > 0.05:  # Minimum amplitude threshold
            processed_segment = audio_preprocessing(segment, fs)
            segments.append(processed_segment)

    return segments

def find_best_segments(all_segments, required_count):
    """Select the most representative segments by energy distribution"""
    if len(all_segments) <= required_count:
        return all_segments
    
    # Calculate energy for each segment
    energies = [np.mean(np.abs(seg)**2) for seg in all_segments]
    
    # Sort by energy and select diverse samples
    sorted_indices = np.argsort(energies)
    selected_indices = np.linspace(0, len(sorted_indices)-1, required_count, dtype=int)
    return [all_segments[i] for i in sorted_indices[selected_indices]]

# Data augmentation

augmentation = Compose([AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    Shift(min_shift=-0.5, max_shift=0.5, p=0.5)
])
def process_bird_species(bird_folders, limit=10):
    """Process bird folders and return up to the specified limit"""
    bird_stats = {}
    
    for bird_name, folder_name in tqdm(bird_folders[:limit], desc="Processing birds"):
        full_folder_path = AUDIO_FILES_DIR / folder_name
        audio_files = list(full_folder_path.glob("*.mp3")) + \
                     list(full_folder_path.glob("*.wav")) + \
                     list(full_folder_path.glob("*.flac"))
        
        if not audio_files:
            print(f"\nWarning: No audio files found for {bird_name}")
            continue

        print(f"\nProcessing {bird_name} ({len(audio_files)} audio files)...")
        
        # Parameters for this bird
        params = {
            'windowSize': windowSize,
            'hopSize': hopSize,
            'NRG_threshold_ratio': NRG_threshold_ratio,
            'segment_samples': segment_samples
        }
        
        # Process all audio files for this bird
        all_segments = []
        for file_path in tqdm(audio_files, desc=f"Processing {bird_name[:15]}...", leave=False):
            audio = safe_mono_loader(file_path, fs)
            if audio is not None:
                segments = extract_birdcall_segments(audio, params)
                all_segments.extend(segments)
        
        if not all_segments:
            print(f"Warning: No segments extracted for {bird_name}")
            continue
            
        # Select best segments
        final_segments = find_best_segments(all_segments, required_segments)
        
        # If we don't have enough, augment with time stretch, shift, and noise
        if len(final_segments) < required_segments:
            needed = required_segments - len(final_segments)
            augmented = []
            source_segments = final_segments if final_segments else all_segments
            for _ in range(needed):
                seg = random.choice(source_segments)
                try:
                    augmented_segment = augmentation(seg, fs)
                    augmented.append(augmented_segment[:segment_samples])
                except Exception:
                    continue
            final_segments.extend(augmented)
        
        # Save segments
        for i, segment in enumerate(final_segments[:required_segments]):
            safe_name = bird_name.replace("/", "-").replace(" ", "_")
            filename = OUTPUT_DIR / f"{safe_name}_{i:03d}.wav"
            ess.MonoWriter(filename=str(filename), format='wav', sampleRate=fs)(segment)
        
        # Record stats
        bird_stats[bird_name] = {
            'total_files': len(audio_files),
            'segments_found': len(all_segments),
            'final_segments': len(final_segments),
            'augmented': max(0, required_segments - len(all_segments))
        }
    
    return bird_stats

def find_all_bird_folders():
    """Find all valid bird folders with audio files"""
    bird_folders = []
    for folder in AUDIO_FILES_DIR.glob("*_sound"):
        if folder.is_dir():
            # Convert folder name to bird name
            bird_name = folder.stem.replace("_sound", "").replace("_", " ").replace("-", "/")
            bird_folders.append((bird_name, folder.name))
    return bird_folders

# Main execution
def main():
    print("Starting Latin American bird audio processing...")

    # 1. Load and filter metadata
    print("\nLoading and filtering metadata...")
    filtered_metadata = load_and_filter_metadata()

    # 2. Get folders for selected species
    bird_folders = get_bird_folders_for_species(filtered_metadata)

    if not bird_folders:
        raise RuntimeError("No valid bird folders found for the selected species!")

    print(f"\nFound {len(bird_folders)} bird species to process:")
    for i, (name, _) in enumerate(bird_folders[:10]):  # Show only the first 10
        print(f"{i+1}. {name}")
    if len(bird_folders) > 10:
        print(f"... and {len(bird_folders)-10} more")

    # 3. Process the species
    stats = process_bird_species(bird_folders)

    # 4. Generate summary
    print("\n" + "="*80)
    print("Processing Summary:")
    print("{:<30} {:<15} {:<15} {:<15} {:<15}".format(
        "Bird Species", "Audio Files", "Segments Found", "Final Segments", "Augmented"))

    for bird, data in stats.items():
        print("{:<30} {:<15} {:<15} {:<15} {:<15}".format(
            bird[:28] + ("..." if len(bird) > 28 else ""),
            data['total_files'],
            data['segments_found'],
            data['final_segments'],
            data['augmented']
        ))

    print(f"\n✅ Successfully processed {len(stats)} Latin American bird species")
    print(f"📁 Output saved to: {OUTPUT_DIR}")

if __name__ == "__main__":
    main()

Already exists: XC201498.mp3 in greater_rhea_sound
Already exists: XC191451.mp3 in greater_rhea_sound
Already exists: XC51833.mp3 in greater_rhea_sound
Already exists: XC49632.mp3 in greater_rhea_sound
Already exists: XC229054.mp3 in greater_rhea_sound
Already exists: XC191450.mp3 in greater_rhea_sound
Already exists: XC54184.mp3 in greater_rhea_sound
Already exists: XC51832.mp3 in greater_rhea_sound
Already exists: XC37806.mp3 in greater_rhea_sound
Already exists: XC686584.mp3 in greater_rhea_sound
Already exists: XC482438.mp3 in greater_rhea_sound
Already exists: XC314181.mp3 in greater_rhea_sound
Already exists: XC229055.mp3 in greater_rhea_sound
Already exists: XC229053.mp3 in greater_rhea_sound
Already exists: XC108649.mp3 in greater_rhea_sound
Already exists: XC54183.mp3 in greater_rhea_sound
Already exists: XC51834.mp3 in greater_rhea_sound
Already exists: XC37804.mp3 in greater_rhea_sound
Already exists: XC510934.mp3 in greater_rhea_sound
Already exists: XC90569.mp3 in lesser_r

Processing birds:   0%|          | 0/10 [00:00<?, ?it/s]


Processing greater_rhea (23 audio files)...


[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping 


Processing grey_tinamou (30 audio files)...


...


Processing cinereous_tinamou (30 audio files)...


[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping 


Processing little_tinamou (30 audio files)...


[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping 


Processing Summary:
Bird Species                   Audio Files     Segments Found  Final Segments  Augmented      
greater_rhea                   23              1995            50              0              
grey_tinamou                   30              9196            50              0              
solitary_tinamou               30              4585            50              0              
black_tinamou                  20              9925            50              0              
great_tinamou                  30              7754            50              0              
white-throated_tinamou         30              3175            50              0              
highland_tinamou               30              11244           50              0              
tawny-breasted_tinamou         30              8816            50              0              
cinereous_tinamou              30              6578            50              0              
little_tinamou               